<a href="https://colab.research.google.com/github/aisutd/Stock-and-ETF-Prediction/blob/main/Stock_Prediction_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://randerson112358.medium.com/stock-market-sentiment-analysis-using-python-machine-learning-5b644f151a3e
# https://www.kaggle.com/aaron7sun/stocknews
# https://medium.com/@kala.shagun/stock-market-prediction-using-news-sentiments-f9101e5ee1f4
# https://medium.com/@kala.shagun/stock-market-prediction-using-news-sentiments-dc4c24c976f7
# https://medium.com/@Currie32/predicting-the-stock-market-with-the-news-and-deep-learning-7fc8f5f639bc
# 

In [ ]:
%pip install vaderSentiment
import pandas_datareader as web
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
import nltk
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

     |████████████████████████████████| 133kB 5.7MB/s 


In [ ]:
df1 = web.DataReader('^DJI', data_source='yahoo', start='2008-08-08', end='2016-07-01')
df1
print(type(df1))
# df2 = pd.read_csv('Combined_News_DJIA.csv')
# df2.head(1)

<class 'pandas.core.frame.DataFrame'>


In [ ]:
headlines = []
for row in range(df2.shape[0]):
  headlines.append(' '.join(str(x) for x in df2.iloc[row,2:27]))
df2["Combined News"] = headlines
#df2.head(3)
updated_df2 = pd.DataFrame(df2, columns=['Date', 'Combined News'])
updated_df2.head(3)

,Date,Combined News
0,2008-08-08,"b""Georgia 'downs two Russian warplanes' as cou..."
1,2008-08-11,b'Why wont America and Nato help us? If they w...
2,2008-08-12,b'Remember that adorable 9-year-old who sang a...


In [ ]:
def sentiment_score(row):
  sid_obj = SentimentIntensityAnalyzer()
  sentence = row
  sentiment_dict = sid_obj.polarity_scores(sentence) 

  #print("Sentence", sentence)
  neg = sentiment_dict['neg']*100
  neu = sentiment_dict['neu']*100
  pos = sentiment_dict['pos']*100
  compound = sentiment_dict['compound']
  #print("Overall sentiment dictionary is : ", sentiment_dict) 
  #print("sentence was rated as ", neg, "% Negative") 
  #print("sentence was rated as ", neu, "% Neutral") 
  #print("sentence was rated as ", pos, "% Positive") 

  # print("Sentence Overall Rated As", end = " ") 
  # if sentiment_dict['compound'] >= 0.05 : 
  #     print("Positive") 
    
  # elif sentiment_dict['compound'] <= - 0.05 : 
  #     print("Negative") 

  # else : 
  #     print("Neutral")
    
  #print("\n")
  return compound, neg, neu, pos

In [ ]:
# combine stock df and news df with the sentiment scores for each row
compound_ = []
negative_ = []
neutral_ = []
positive_ = []

for i in range(len(updated_df2)):
  curr = updated_df2['Combined News'][i]
  compound, neg, neu, pos = sentiment_score(curr)
  #print(compound, neg,neu,pos, '\n')
  compound_.append(compound)
  negative_.append(neg)
  neutral_.append(neu)
  positive_.append(pos)


In [ ]:
print(len(df1) == len(compound_) == len(negative_) == len(neutral_) == len(positive_))

True


In [ ]:
df1['Compound'] = compound_
df1['Negative'] = negative_
df1['Neutral'] = neutral_
df1['Positive'] = positive_
df1.tail()

,High,Low,Open,Close,Volume,Adj Close,Compound,Negative,Neutral,Positive
Date,,,,,,,,,,
2016-06-27,17355.210938,17063.080078,17355.210938,17140.240234,1387400,17140.240234,-0.9644,14.8,75.8,9.4
2016-06-28,17409.720703,17190.509766,17190.509766,17409.720703,1121900,17409.720703,-0.9571,13.2,76.7,10.2
2016-06-29,17704.509766,17456.019531,17456.019531,17694.679688,1063800,17694.679688,-0.9975,22.5,68.4,9.1
2016-06-30,17930.609375,17711.800781,17712.759766,17929.990234,1330300,17929.990234,-0.9977,20.2,73.8,6.1
2016-07-01,18002.380859,17916.910156,17924.240234,17949.369141,821600,17949.369141,-0.9983,21.2,72.9,5.9


In [ ]:
# Add column in df1 where it tells whether the stock went up or down from the previous day
up_or_down_tomorrow = []
for i in range(0, len(df1)-1):
  today = df1['Close'][i]
  tomorrow = df1['Close'][i+1]
  if tomorrow > today:
    up_or_down_tomorrow.append(1)
  else:
    up_or_down_tomorrow.append(0)
up_or_down_tomorrow.append(0)
print(len(up_or_down_tomorrow))
print(len(df1))

1989
1989


In [ ]:
df1['Up(1) or Down(0) Tomorrow'] = up_or_down_tomorrow
df1.head()

,High,Low,Open,Close,Volume,Adj Close,Compound,Negative,Neutral,Positive,Up(1) or Down(0) Tomorrow
Date,,,,,,,,,,,
2008-08-08,11759.959961,11388.040039,11432.089844,11734.320312,2128300,11734.320312,-0.9979,22.9,72.3,4.9,1
2008-08-11,11867.110352,11675.530273,11729.669922,11782.349609,1831900,11782.349609,-0.9804,17.0,74.9,8.1,0
2008-08-12,11782.349609,11601.519531,11781.700195,11642.469727,1735900,11642.469727,-0.9658,12.2,82.3,5.6,0
2008-08-13,11633.780273,11453.339844,11632.809570,11532.959961,1825500,11532.959961,-0.9809,14.5,79.0,6.5,1
2008-08-14,11718.280273,11450.889648,11532.070312,11615.929688,1597900,11615.929688,-0.9802,16.7,73.8,9.5,1


In [ ]:
import sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report

x = np.array(df1.drop(['Up(1) or Down(0) Tomorrow'],1))
y = df1['Up(1) or Down(0) Tomorrow']
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2)
model = LinearDiscriminantAnalysis().fit(x_train, y_train)
predictions = model.predict(x_test)
predictions

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,

In [ ]:
print( classification_report(y_test, predictions) )

              precision    recall  f1-score   support

           0       0.50      0.21      0.30       176
           1       0.57      0.83      0.68       222

    accuracy                           0.56       398
   macro avg       0.54      0.52      0.49       398
weighted avg       0.54      0.56      0.51       398

